# CLIP as feature extractor + XGBoost classification




CLIP encoder has very strong feature extraction capabilities. In pair with XGBoost it is fast and effective way to classify small datasets.
 

In [1]:
from torchvision import transforms
import numpy as np
import os
import sys
sys.path.append(os.path.abspath(".."))  
from models.ui_dataset import UIDataset
from torch.utils.data import random_split, DataLoader
import torch
import wandb
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score, recall_score, precision_score
from wandb.integration.xgboost import WandbCallback
import torch
import os
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import CLIPModel, CLIPFeatureExtractor
from PIL import Image
from tqdm import tqdm

In [2]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: urik-voevidka (urik-voevidka-ukrainian-catholic-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
dataset_path = "/Users/yuriivoievidka/UCU/CV/cv-project/data"

In [4]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
model_id = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(model_id).to(device)
feature_extractor = CLIPFeatureExtractor.from_pretrained(model_id)  

/Users/yuriivoievidka/.pyenv/versions/3.12.7/envs/torch/lib/python3.12/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [5]:
class UIDataset(Dataset):
    def __init__(self, root_dir, feature_extractor=None):
        self.root_dir = root_dir
        self.feature_extractor = feature_extractor
        self.image_paths = []

        for label in os.listdir(self.root_dir):
            subdir_path = os.path.join(self.root_dir, label)
            if os.path.isdir(subdir_path):
                for filename in os.listdir(subdir_path):
                    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff', '.webp')):
                        file_path = os.path.join(subdir_path, filename)
                        self.image_paths.append((file_path, label))  # Store (image path, label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        file_path, label = self.image_paths[index]

        image = Image.open(file_path).convert("RGB")  
        
        if self.feature_extractor:
            image = self.feature_extractor(images=image, return_tensors="pt")["pixel_values"].squeeze(0)

        return image, label


In [6]:
dataset_path = "/Users/yuriivoievidka/UCU/CV/cv-project/data"
ui_dataset_all = UIDataset(root_dir=dataset_path, feature_extractor=feature_extractor)

ui_train_size = int(len(ui_dataset_all) * 0.6)
ui_val_size = int(len(ui_dataset_all) * 0.2)
ui_test_size = len(ui_dataset_all) - ui_train_size - ui_val_size
train_dataset, val_dataset, test_dataset = random_split(ui_dataset_all, [ui_train_size, ui_val_size, ui_test_size])

ui_dataset_all_dataloader = DataLoader(ui_dataset_all, batch_size=32, shuffle=True)


In [7]:
with torch.no_grad():
    all_embeddings = []
    for batch in tqdm(ui_dataset_all_dataloader):
        images, labels = batch
        images = images.to(device)

        embeddings = clip_model.get_image_features(images)

        embeddings /= embeddings.norm(dim=-1, keepdim=True)

        all_embeddings.append(embeddings.cpu())

    all_embeddings = torch.cat(all_embeddings, dim=0)

print("Embeddings shape:", all_embeddings.shape)

 12%|█▏        | 16/130 [00:16<01:59,  1.05s/it]/Users/yuriivoievidka/.pyenv/versions/3.12.7/envs/torch/lib/python3.12/site-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 130/130 [02:13<00:00,  1.03s/it]

Embeddings shape: torch.Size([4149, 512])


In [9]:
all_embeddings_np = all_embeddings.numpy()

label_to_index = {label: idx for idx, label in enumerate(set(label for _, label in ui_dataset_all.image_paths))}
all_labels_np = np.array([label_to_index[label] for _, label in ui_dataset_all.image_paths])

np.save("clip_embeddings.npy", all_embeddings_np)
np.save("clip_labels.npy", all_labels_np)

In [10]:
!export WANDB_API_KEY="7e54b169f751025afccd3ef14aacaf584f797cb1"

In [8]:
embeddings = np.load("clip_embeddings.npy")  
labels = np.load("clip_labels.npy")  

X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.3, random_state=42)

In [9]:
sweep_config = {
    "method": "random", 
    "metric": {
      "name": "accuracy",
      "goal": "maximize"   
    },
    "parameters": {
        "booster": {
            "values": ["gbtree","gblinear"]
        },
        "max_depth": {
            "values": [3, 6, 9, 12]
        },
        "learning_rate": {
            "values": [0.1, 0.05, 0.2]
        },
        "subsample": {
            "values": [1, 0.5, 0.3]
        }
    }
}

In [10]:
sweep_id = wandb.sweep(sweep_config, project="ui-classification-experiments")

Create sweep with ID: 9rkuh7kp
Sweep URL: https://wandb.ai/urik-voevidka-ukrainian-catholic-university/ui-classification-experiments/sweeps/9rkuh7kp


In [11]:
def train():
  config={
        "learning_rate": 0.02,
        "architecture": "XGBoost",
        "dataset": "DesktopUI",
        "epochs": 100,
        "model": "XGBoost",
        "test_size": 0.3,
        "random_state": 42
    }

  wandb.init(config=config)  # defaults are over-ridden during the sweep
  config = wandb.config

  model = xgb.XGBClassifier(objective="multi:softmax", booster=config.booster, max_depth=config.max_depth,
                        learning_rate=config.learning_rate, subsample=config.subsample)
  model.fit(X_train, y_train)

  y_pred = model.predict(X_test)
  predictions = [round(value) for value in y_pred]

  accuracy = accuracy_score(y_test, predictions)
  
  f1 = f1_score(y_test, predictions, average="weighted")
  recall = recall_score(y_test, predictions, average="weighted")
  precision = precision_score(y_test, predictions, average="weighted")

  wandb.log({
      "accuracy": accuracy,
      "f1_score": f1,
      "recall": recall, 
      "precision": precision,
      "classification_report": classification_report(y_test, predictions, output_dict=True)
  })
  

In [12]:
wandb.agent(sweep_id, train, count=25)

wandb: Agent Starting Run: bxw5a2ia with config:
wandb: 	booster: gblinear
wandb: 	learning_rate: 0.05
wandb: 	max_depth: 9
wandb: 	subsample: 0.3


[15:25:37] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "max_depth", "subsample" } are not used.



accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.44096
f1_score,0.42114
precision,0.41561
recall,0.44096


wandb: Agent Starting Run: py7hh5nn with config:
wandb: 	booster: gblinear
wandb: 	learning_rate: 0.05
wandb: 	max_depth: 9
wandb: 	subsample: 0.3


[15:25:42] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "max_depth", "subsample" } are not used.



...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: t65tq0jj with config:
wandb: 	booster: gbtree
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 6
wandb: 	subsample: 0.3


accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.45141
f1_score,0.42546
precision,0.4027
recall,0.45141


wandb: Agent Starting Run: 5151by7v with config:
wandb: 	booster: gblinear
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 9
wandb: 	subsample: 0.5


[15:29:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "max_depth", "subsample" } are not used.



accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.43293
f1_score,0.41927
precision,0.41268
recall,0.43293


In [13]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
classes = ["clean-ui", "to-crop", "unnecessary"]

NameError: name 'model' is not defined

In [14]:
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=classes))

Classification Report:


NameError: name 'y_pred' is not defined